In [1]:
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Reusable classes
import sys
sys.path.append("/home/jovyan/work/upload/ml-experiments/sc_wrangling")

# Data path
import pathlib
FOLDER = pathlib.Path("/home/jovyan/work/upload/")
TABLE_PATH = FOLDER / 'data/pickle/df_columns_labels.pkl'

# Manage ML lifecycle
import mlflow
from mlflow import log_params, set_tags, log_metrics
MLFLOW_TRACKING_URI = 'http://srv.meaningfy.ws:8989'

# Math stuff
import numpy as np

# Data visualisation
import pandas as pd

# Vectorize algotrithm
from gensim.models import Word2Vec

# Support Vector Classifier Algorithm
from sklearn.neighbors import KNeighborsClassifier as KNC
# :n_neighbors: represents the number of neighbors to use for k-neighbors queries
N_NEIGHBORS=3
P=1

# Intermediate steps of the pipeline must be ‘transforms’,
# that is, they must implement fit and transform methods.
from sklearn.pipeline import Pipeline

# Splitting data into train and test
from sklearn.model_selection import train_test_split
RANDOM_STATE = 42
# Where "train" contains 70% of data and "test" - 30%
TEST_SIZE = 0.3
SHUFFLE = True

# Mean embedding function
from mean_vectorizer import MeanEmbeddingVectorizer

# Metrics Evaluation Methods
from evaluation_metrics import model_evaluation_metrics

# Transform DataFrame to dictionary
from dictionary_transformation import series_pair_to_dict

In [2]:
# Read data
df = pd.read_pickle(TABLE_PATH)
df.head()

,Concatenated Data (clean),Category,Subcategory,Type of measure,Target groups,One person or microenterprises|Self-employed|Solo-self-employed,SMEs|Sector specific set of companies,One person or microenterprises|SMEs,Employees in standard employment|Other groups of workers,Employees in standard employment|Larger corporations,...,Children (minors)|Disabled|Older citizens|Parents|SMEs|Single parents|Single parents in employment|The COVID-19 risk group|Workers in care facilities|Workers in essential services,Employees in standard employment|Workers in care facilities|Workers in non-standard forms of employment,Companies providing essential services|Workers in essential services,Contractors of a company,Other businesses|Unemployed,Seasonal workers|Workers in non-standard forms of employment,Employees in standard employment|Particular professions,Businesses,Citizens,Workers
0,"[hardship, case, fund, safety, net, selfemploy...",2,12,0,One person or microenterprises|Self-employed|S...,1,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
1,"[state, support, tourism, access, financeas, t...",0,0,0,SMEs|Sector specific set of companies,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,"[bank, guarantees, smes, oneperson, enterprise...",0,0,0,One person or microenterprises|SMEs,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
3,"[emergency, measures, relating, shorttime, wor...",6,14,0,Employees in standard employment|Other groups ...,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,1
4,"[airbus, agreement, making, unworked, hours, p...",3,7,2,Employees in standard employment|Larger corpor...,0,0,0,0,1,...,0,0,0,0,0,0,0,1,1,1


In [3]:
# Independent data
columns = df['Concatenated Data (clean)']
# Label data
target_groups_subclasses = df[df.columns[5:208]]


## word2vec model


In [4]:
# Insert our data and set minimal word count to 10, and size of each word to 300 vectors
model = Word2Vec.load('/home/jovyan/work/Dan/data/pwdb/word2vec/df.model')
w2v_dict = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


In [5]:
# Use pipes to implement steps of fit and transform method
knn_w2v = Pipeline([
    # Add the words we want to mean
    ("word2vec vectorizer", MeanEmbeddingVectorizer(w2v_dict)),
    # Use KNN algorithm
    # :n_neighbors: represents the number of neighbors to use for k-neighbors queries
    ("KNN", KNC(n_neighbors=N_NEIGHBORS, p=P))])

## Train SVM Model

In [6]:
# Split data into test and train sets
# Where "train" contains 70% of data and "test" - 30%
X_train, X_test, y_train, y_test = train_test_split(
    columns, target_groups_subclasses, random_state=RANDOM_STATE, test_size=TEST_SIZE, shuffle=SHUFFLE)

In [8]:
# Fit KNN model with out train data
knn_w2v.fit(X_train, y_train)


Pipeline(steps=[('word2vec vectorizer',
                 <mean_vectorizer.MeanEmbeddingVectorizer object at 0x7fc5b5534460>),
                ('KNN', KNeighborsClassifier(n_neighbors=3, p=1))])

In [9]:
# Model Evaluation
knn_w2v.score(X_train, y_train)

0.23853211009174313

In [10]:
# KNN prediction based on test data
prediction = knn_w2v.predict(X_test)
prediction

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

## Evaluation

In [11]:
evaluation = model_evaluation_metrics(y_test, prediction, 'Target groups')

In [12]:
# Transform columns data to dict to use in MLFlow
transformation = series_pair_to_dict(evaluation, 'Evaluation Metrics', 'Target groups')

In [13]:
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(experiment_name="Target group subclasses (Word2Vec)")

with mlflow.start_run():

    parameters = {"Language model": 'word2vec',
                  "Random state": RANDOM_STATE,
                  "Test size": TEST_SIZE,
                  "Shuffle": SHUFFLE,
                  "n_neighbors": N_NEIGHBORS,
                  "p": P}
    log_params(parameters)

    log_metrics(transformation)